<a href="https://colab.research.google.com/github/imp-etus/Education-Inequality/blob/main/Data%20Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

The purpose of this notebook is to prepare demographic data relevant to education (income, %college attended, and so on) for analysis later on. Additionally, this notebook uses MICE imputation to replace missing values & 

## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

# Train-test splits
from sklearn.model_selection import train_test_split

# Model preprocessing
from sklearn.preprocessing import StandardScaler

# Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

## The data

This project utilizes two data sets. The primary data set is the EdGap data set from [EdGap.org](https://www.edgap.org/#5/37.875/-96.987). This data set from 2016 includes information about average ACT or SAT scores for schools and several socioeconomic characteristics of the school district. The secondary data set is basic information about each school from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp).





### EdGap data

All socioeconomic data (household income, unemployment, adult educational attainment, and family structure) are from the Census Bureau's American Community Survey. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) report that ACT and SAT score data is from each state's department of education or some other public data release. The nature of the other public data release is not known.

The quality of the census data and the department of education data can be assumed to be reasonably high. 

[EdGap.org](https://www.edgap.org/#5/37.875/-96.987) do not indicate that they processed the data in any way. The data were assembled by the [EdGap.org](https://www.edgap.org/#5/37.875/-96.987) team, so there is always the possibility for human error. Given the public nature of the data, we would be able to consult the original data sources to check the quality of the data if we had any questions.

### School information data

The school information data is from the [National Center for Education Statistics](https://nces.ed.gov/ccd/pubschuniv.asp). This data set consists of basic identifying information about schools and can be assumed to be of reasonably high quality. As for the EdGap.org data, the school information data is public, so we would be able to consult the original data sources to check the quality of the data if we had any questions.


## Load the data

Load the EdGap
 data set

In [ ]:
edgap = pd.read_excel('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/education/EdGap_data.xlsx')

/usr/local/lib/python3.9/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Load the school information data

In [ ]:
!wget https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0

--2023-04-24 06:46:25--  https://www.dropbox.com/s/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv [following]
--2023-04-24 06:46:25--  https://www.dropbox.com/s/raw/lkl5nvcdmwyoban/ccd_sch_029_1617_w_1a_11212017.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc06f422f212eacb452f1d5b3444.dl.dropboxusercontent.com/cd/0/inline/B6yO9D5jJT9Qdudor9DPQJbltYHtSWIMrHq0f4t_L4ABp6qyb3K_UB8_aQ26g-gz_bw3xE9KQgTNZBfQ3LpNWfh0CD1uEkMwYstLgdBop7wgi0T6yGF9oGr1ZT4Ky_ttDvtCG84zIb1NZTj7vEudhTqmULnPH5PytwjtSiIO8yUpZw/file# [following]
--2023-04-24 06:46:26--  https://uc06f422f212eacb452f1d5b3444.dl.dropboxusercontent.com/cd/0/inline/B6yO9D

In [ ]:
school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')

<ipython-input-5-da1a7837c049>:1: DtypeWarning: Columns (6,9,14,15,18,19,21,22,25,26,29,31,35,39,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  school_info = pd.read_csv('ccd_sch_029_1617_w_1a_11212017.csv?dl=0', encoding='unicode_escape')


## Convert data types, if necessary

This step converts the ID section of the school info to ints (because floating-point math is messy) and drops all schools that won't match to general school ID.

In [ ]:
school_info = school_info.dropna(subset=['NCESSCH'])
school_info['NCESSCH'].astype('int64')

0          10000200277
1          10000201667
2          10000201670
3          10000201705
4          10000201706
              ...     
102178    780003000025
102179    780003000026
102180    780003000027
102181    780003000033
102182    780003000034
Name: NCESSCH, Length: 102181, dtype: int64

## Are the data suitable for answering the question?

We want to perform quick exploratory data analysis to determine whether the data are sufficient to answer our question. If the data are not sufficient, we do not want to waste time doing anything that will not be productive.

## Select relevant subsets of the data

:The school information data set contains a lot of information. We only need the year, school identity, location, and school type information.

Keep the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL`

## Rename columns

Rename columns with spaces to proper case & drop all extraneous info.

In [ ]:
edgap = edgap.rename(columns={"NCESSCH School ID":"id", 
              "CT Pct Adults with College Degree":"percent_college",        
              "CT Unemployment Rate":"rate_unemployment", 
              "CT Pct Childre In Married Couple Family":"percent_married",
              "CT Median Household Income":"median_income",
              "School ACT average (or equivalent if SAT score)":"average_act",
              "School Pct Free and Reduced Lunch":"percent_lunch"})

In [ ]:
school_info = school_info[{'SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL'}]
school_info = school_info.rename(columns = {"SCHOOL_YEAR":"year", "NCESSCH":"id", "MSTATE":"state", "MZIP":"zip_code", "SCH_TYPE_TEXT":"school_type", "LEVEL":"school_level"})

<ipython-input-8-f12e5bed84b7>:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  school_info = school_info[{'SCHOOL_YEAR', 'NCESSCH', 'MSTATE', 'MZIP', 'SCH_TYPE_TEXT', 'LEVEL'}]


Rename the columns `SCHOOL_YEAR`, `NCESSCH`, `MSTATE`, `MZIP`, `SCH_TYPE_TEXT`, `LEVEL` to `year`, `id`, `state`, `zip_code`, `school_type`, `school_level`

## Join data frames 

Joining onto edgap b/c we don't care about any data points in school_info that don't have educational data.

In [ ]:
df_merged = edgap.merge(school_info, on='id', how='left')

## Quality Control

Check for out-of-range values or values that do not match what we want to analyze. Either set values to `NaN` or remove the observations, as appropriate.

In [ ]:
df_merged.agg(['min', 'max']).round(2)

<ipython-input-10-4f489b047f1c>:1: FutureWarning: ['state', 'school_level', 'school_type', 'year', 'zip_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_merged.agg(['min', 'max']).round(2)


,id,rate_unemployment,percent_college,percent_married,median_income,average_act,percent_lunch
min,100001600143,0.00,0.09,0.0,3589.0,-3.07,-0.05
max,560583000335,0.59,1.00,1.0,226181.0,32.36,1.00


Spot all the places where the average ACT score is negative or the % of students getting school lunches is negative.

In [ ]:
sub_one_act = np.where(df_merged['average_act'] < 1.0)[0]
subzero_percent_lunch = np.where(df_merged['percent_lunch'] < 0.0)[0]

Drop invalid ACT scores & blank invalid "percent getting school lunch" scores.

In [ ]:
df_merged = df_merged.drop(sub_one_act)
df_merged.loc[subzero_percent_lunch, 'percent_lunch'] = np.nan
df_merged = df_merged.loc[df_merged['school_level'] == 'High']

## Train test split

We will use the test set approach to compare models predicting `average_act`. 

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bb/ML_dataset_training_validation_test_sets.png/800px-ML_dataset_training_validation_test_sets.png)

Split the data into training and testing sets. Keep 20% of the data for the test set.

In [ ]:
x = df_merged[df_merged.columns.difference(['id', 'average_act'])]
y = df_merged['average_act']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state = 1)
print(x_train.shape, x_test.shape)

(5781, 10) (1446, 10)


In [ ]:
x_train.agg(['mean', 'std']).round(2)

<ipython-input-42-1007bccc90f3>:1: FutureWarning: ['school_level', 'school_type', 'state', 'year', 'zip_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x_train.agg(['mean', 'std']).round(2)


,median_income,percent_college,percent_lunch,percent_married,rate_unemployment
mean,53170.26,0.57,0.41,0.64,0.10
std,24827.94,0.17,0.23,0.19,0.06


In [ ]:
x_test.agg(['mean', 'std']).round(2)

<ipython-input-43-e6765bcb6b0e>:1: FutureWarning: ['school_level', 'school_type', 'state', 'year', 'zip_code'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  x_test.agg(['mean', 'std']).round(2)


,median_income,percent_college,percent_lunch,percent_married,rate_unemployment
mean,51179.23,0.57,0.42,0.63,0.10
std,22358.47,0.16,0.24,0.19,0.06


## Data imputation

Using MICE imputation to replace missing values in the columns corresponding to predictor variables in the analysis.

In [ ]:
from sklearn import preprocessing

In [ ]:
imputer = IterativeImputer()

In [ ]:
imputer.fit(x_train.loc[:,'median_income':'rate_unemployment'])

IterativeImputer()

In [ ]:
x_train.loc[:, 'median_income':'rate_unemployment'] = imputer.transform(x_train.loc[:, 'median_income':'rate_unemployment'])

In [ ]:
x_train.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

In [ ]:
x_test.isna().sum()

median_income        0
percent_college      0
percent_lunch        0
percent_married      0
rate_unemployment    0
school_level         0
school_type          0
state                0
year                 0
zip_code             0
dtype: int64

In [ ]:
x_test.loc[:, 'median_income':'rate_unemployment'] = imputer.transform(x_test.loc[:, 'median_income':'rate_unemployment'])

In [ ]:
df_train = x_train.join(y_train)

In [ ]:
df_test = x_test.join(y_test)

## Create relevant derived variables as new columns

(potentially updated if analysis sections bear fruit)

## Export the clean .csv files

Export files with the training and testing data

In [ ]:
from google.colab import files

In [ ]:
df_train.to_csv('cleaned_training.csv', encoding='utf-8-sig', index=False)
df_test.to_csv('cleaned_testing.csv', encoding='utf-8-sig', index=False)

In [ ]:
files.download('cleaned_training.csv')
files.download('cleaned_testing.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>